This notebook shows how to create a Threshold Crossing Event (TCE) object using the framework built into exovetter. 

A TCE stores the measured properties (orbital period, transit depth, etc.) of a proposed transit that usually come out of a transit search pipeline. You can create TCEs by manually inputting the data, or automattically populating the required fields using exo.mast's API. 


In [1]:
%load_ext autoreload
%autoreload 2

## Example: Creating a TCE in TESS BKJD by manually inputing properties:

In [2]:
from astropy import units as u
from exovetter import const as exo_const
from exovetter.tce import Tce

# Define properties which will be used in the TCE
period = 5.3 * u.day
epoch = 133.4 * u.day
depth = 1 * exo_const.ppm
duration = 24 * u.hr

# create a TCE object using the exovetter.tce.Tce class
my_tce = Tce(period=period, epoch=epoch, 
             epoch_offset=exo_const.bkjd, depth=depth, 
             duration=duration, comment='example')

display(my_tce)

# TCEs are then used as an input to any of the vetters
# You can check if all required parameters are present to run in a vetter using the validate method
if my_tce.validate() is True:
    print('All required parameters are present')
else:
    print('All required parameters are not present')
    

{'period': <Quantity 5.3 d>,
 'epoch': <Quantity 133.4 d>,
 'epoch_offset': <Quantity -2454833. d>,
 'depth': <Quantity 1.e-06>,
 'duration': <Quantity 24. h>,
 'comment': 'example'}

All required parameters are present


## Example: Creating a TCE from archival data

For objects in the exo.MAST archive (https://exo.mast.stsci.edu/docs/) we can pull down data to populate the TCE object.

In [11]:
import requests

# You can obtain a json dictionary of planet properties using the exo.MAST api
masturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/identifiers/"
myparams = {"name":"WASP-18 b"}
header={} # you can add HTTP headers to a request

r = requests.get(url = masturl, params = myparams, headers = header)
planet_names = r.json()

planet_names

{'canonicalName': 'WASP-18 b',
 'starName': 'WASP-18',
 'ra': 24.354208334287005,
 'dec': -45.677930555343636,
 'planetNames': ['CPC 0 759 b',
  'UBV 1689 b',
  'HIP 7562 b',
  'PPM 306061 b',
  '2MASS J01372503-4540404 b',
  'TYC 8040-72-1 b',
  'WASP-18 b',
  'SAO 215585 b',
  'GEN# +1.00010069 b',
  '[CS62] E1 13 b',
  'HIC 7562 b',
  'Gaia DR1 4955371363037611136 b',
  'CPD-46 168 b',
  'CD-46 449 b',
  'GSC 08040-00072 b',
  'HD 10069 b'],
 'keplerID': None,
 'keplerTCE': None,
 'tessID': 100100827,
 'tessTCE': 'TCE_2'}

In [13]:
masturl = "https://exo.mast.stsci.edu/api/v0.1/exoplanets/WASP-18 b/properties/"
header={} # you can add HTTP headers to a request

r = requests.get(url = masturl, headers = header)
planet_names = r.json()

planet_names

[{'canonical_name': 'WASP-18 b',
  'exoplanetID': 5169,
  'catalog_name': 'exoplanets.org',
  'planet_name': 'WASP-18 b',
  'disposition': 'confirmed planet',
  'modified_date': '2023-10-10 02:47:09.758000',
  'star_name': 'WASP-18',
  'component': 'b',
  'Rs': 1.216,
  'Rs_unit': 'R_sun',
  'Rs_upper': 0.067,
  'Rs_lower': 0.0539999999999999,
  'Rs_ref': 'Hellier 2009',
  'Rs_url': 'http://adsabs.harvard.edu/abs/2009Natur.460.1098H',
  'Ms': 1.22,
  'Ms_unit': 'M_sun',
  'Ms_upper': 0.03,
  'Ms_lower': 0.03,
  'Ms_ref': 'Enoch 2010',
  'Ms_url': 'http://adsabs.harvard.edu/abs/2010A%26A...516A..33E',
  'Fe/H': 0.0,
  'Fe/H_upper': 0.09,
  'Fe/H_lower': 0.09,
  'Fe/H_ref': 'Hellier 2009',
  'Fe/H_url': 'http://adsabs.harvard.edu/abs/2009Natur.460.1098H',
  'stellar_gravity': 4.367,
  'stellar_gravity_upper': 0.028,
  'stellar_gravity_lower': 0.042,
  'stellar_gravity_ref': 'Hellier 2009',
  'stellar_gravity_url': 'http://adsabs.harvard.edu/abs/2009Natur.460.1098H',
  'Teff': 6400.0,
  '

In [ ]:
# ticid = planet_names['tessID']
# tceid = planet_names['tessTCE']


This process can be automated using the utils function `get_mast_tce()` which populates the TCE with the period, epoch, epoch_offset, depth, and duration from exo.MAST. If multiple catalogs are present it will provide a list of TCEs created from each catalog.

In [2]:
from exovetter import utils

#Name of a TOI Example
candidate = "WASP-18 b"
tce = utils.get_mast_tce(candidate)

tce

<Response [502]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

To create model transits from a `~exovetter.tce.Tce`, see the
`exovetter.model` module. For example, you can obtain flux from a boxcar
model using :func:`~exovetter.model.create_box_model_for_tce`.